In [1]:
cd ..

/home/jovyan/Robbi/dea-intertidal


In [ ]:
pip install odc-geo --quiet

In [34]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import numpy as np
import odc.geo.xr
import datacube

from intertidal.extents import load_data

dc = datacube.Datacube(app='Intertidal_elevation')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load data
For each Area of Interest, lazily load datacube data:

In [23]:
# Load AOI vector regions
aoi_gdf = gpd.read_file('data/raw/Phase_1_NT_Gov.geojson').set_index('Name').to_crs('EPSG:3577')

# Load data for each ROI
aois = ['Wollogorang', 'McCarthur River', 'Greenbank', 'Seven Emu']
geoboxes = []

for aoi in aois:
    
    # Import region of interest    
    geom = Geometry(geom=aoi_gdf.loc[aoi].geometry, crs='EPSG:3577')
    
    # Load data and append geobox
    ds = load_data(dc, geom)
    geoboxes.append(ds.odc.geobox)


### Create template raster
Create new GeoBox from the union of all inputs:

In [25]:
geobox_union = odc.geo.geobox.geobox_union_conservative(geoboxes=geoboxes)

Create and save template raster:

In [37]:
ds_union = odc.geo.xr.xr_zeros(geobox=geobox_union).astype(np.int16)
ds_union.odc.write_cog('intertidal_template.tif', overwrite=True)

### Reproject LiDAR DEM into template raster grid
```
gdalwarp -s_srs EPSG:28353 -t_srs EPSG:32753 -dstnodata -9999.0 -tr 10.0 10.0 -r average -te 640270.0 8165330.0 820370.0 8276390.0 -te_srs EPSG:32753 -multi -ot Float32 -of GTiff -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9 "dem.vrt" "lidar_10m.tif"
```